In [ ]:
import os
import re
import time
import csv
import random
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from datetime import date, timedelta, datetime
import datetime
from time import sleep

### DEMO

# a.	Success to scrape data (10 points)

In [ ]:
browser = webdriver.Chrome(executable_path = '/Users/alita/Desktop/chromedriver_win32/chromedriver')

URL = 'https://seekingalpha.com/article/4392655-sarepta-therapeutics-inc-srpt-presents-evercore-isi-healthconx-conference-transcript'
page1 = browser.get(URL)
html = browser.page_source
soup = BeautifulSoup(html, "lxml")
participants = []
conf_article = soup.find_all('article')

for conf_elem in conf_article:
    try:
        title = conf_elem.find('h1').text
    except:
        title = 'none'
    try:
        txt = conf_elem.find('div', class_='sa-art article-width').next_element.text
        txt = re.sub("\s\s+" , " ", txt)
    except:
        txt = 'none'
        
    try: 
        ticker = re.search('\((.+?):(.+?)\)', txt).group(2)
    except:
        ticker = 'none'
    try:
        company_name = re.search('(.+?)\((.+?):', txt).group(1)
    except:
        company_name = 'none'
    try:
        date_time_str = re.search('Call (.+?) ET', txt).group(1)
        try:
            date_time_obj = datetime.datetime.strptime(date_time_str, '%B %d, %Y %I:%M %p')
        except:
            date_time_obj = 'none'
        try:
            date = date_time_obj.date()
        except:
            date = '00-00-00'
        try:
            time = date_time_obj.time()
        except:
            time = '00:00:00'
    except:
        date = '00-00-00'
        time = '00:00:00'
        
    # get participants
    
comp_participant = soup.find('p', {'class':'p p1'}, text = 'Company Participants')
# print('Company Participants: ')
for sibling in comp_participant.next_siblings:
    try:
        if(sibling.text == 'Conference Call Participants'):
            break
        p = sibling.text
        name = re.search('(.+?) - (.+?)', p).group(1)
        type_ = re.search('(.+?) - (.+)', p).group(2)
        organization = company_name
        participants.append({'participant_name': name, 'participant_type': type_, 'participant_organization': organization, 'conference_title': title})
    except:
        continue
# print('Conference Call Participants: ')    
conf_participant = soup.find('p', {'class':'p p1'}, text = 'Conference Call Participants')
for sibling in conf_participant.next_siblings:
    try:
        p = sibling.text
        if(sibling.find('strong')):
            now_speech = sibling
            break
        name = re.search('(.+?) - (.+?)', p).group(1)
        organization = re.search('(.+?) - (.+)', p).group(2)
        type_ = 'guest participant'
        participants.append({'participant_name': name, 'participant_type': type_, 'participant_organization': organization})
    except:
        continue
#print(participants)

#print(now_speech)
speech = []
speaker = now_speech.text
section = 'Presentation'
spoke = " "
spoken_text = []
# get speech
for sibling in now_speech.next_siblings:
    try:
        if(sibling.find('strong')):
            print(spoken_text)
            spoke = ''.join(spoken_text)
            speech.append({'speech_section': section, 'speech_text': spoke, 'speech_partcipant': speaker})
            spoken_text = []
            if(sibling.text == 'Question-and-Answer Session'):
                section = 'Question-and-Answer'
                continue
            speaker = sibling.text
            continue
        spoken_text.append(sibling.text)
    except:
        continue

# b.	Success to extract names and affiliations of participants

In [ ]:
print(participants)

# c.	Success to extract speech for each participant. (10 points)

In [ ]:
print(speech)

# START SCRAPING FOR DATA BASE

In [ ]:
def write_to_file(links):
    with open('links.txt', 'a') as f:
        f.writelines(links)
browser = webdriver.Chrome(executable_path = '/Users/alita/Desktop/chromedriver_win32/chromedriver')

URL1 = 'https://seekingalpha.com/earnings/earnings-call-transcripts'
URLS = []
URLS.append(URL1)
for i in range(3,6):
    URL1 = "https://seekingalpha.com/earnings/earnings-call-transcripts/" + str(i)
    URLS.append(URL1)
   
links = []

for URL in URLS: 
    page1 = browser.get(URL)
    html = browser.page_source
    soup = BeautifulSoup(html, "lxml")
    for link in soup.find_all('a', class_='dashboard-article-link'):
        link_url = link.get('href')
        if 'earnings-call-transcript' in link_url:
            links.append('https://seekingalpha.com' + link_url + '\n')
write_to_file(links)

In [ ]:
def write_to_company(row):
    with open('company.csv', mode='w') as csv_file:
        fieldnames = ['cy_ticker', 'cy_name']
        writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
        for data in row:
            writer.writerow(data)
        
def write_to_conference(row):
    with open('conference.csv', mode='w') as csv_file:
        fieldnames = ['cf_id', 'cf_title', 'cf_date', 'cf_time', 'Q', 'cf_year', 'cy_ticker']
        writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
        for data in row:
            writer.writerow(data)

def write_to_participant(row):
    with open('participant.csv', mode='w') as csv_file:
        fieldnames = ['p_name', 'p_type', 'p_org', 'cf_id']
        writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
        for data in row:
            writer.writerow(data)

def write_to_speech(row):
    with open('speech.csv', mode='w') as csv_file:
        fieldnames = ['speech_id', 'speech_section', 's_text', 'p_name', 'cf_id']
        writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
        for data in row:
            writer.writerow(data)            


In [ ]:
#using Chromedriver to open webpages
browser = webdriver.Chrome(executable_path = '/Users/alita/Desktop/chromedriver_win32/chromedriver')

companies = []
conferences = []
participants = []
speech = []
spch_id = 11110

file1 = open('links.txt', 'r') 

for line in file1:   
    URL = line
    conf_id = re.search('https://seekingalpha.com/article/(.+?)-', URL).group(1)
    page1 = browser.get(URL)
    sleep(4)
    html = browser.page_source
    soup = BeautifulSoup(html, "lxml")

    try: 
        conf_article = soup.find_all('article')

        for conf_elem in conf_article:
            try:
                title = conf_elem.find('h1').text
            except:
                title = 'none'
            try:
                txt = conf_elem.find('div', class_='sa-art article-width').next_element.text
                txt = re.sub("\s\s+" , " ", txt)
            except:
                txt = 'none'

        
            try: 
                q = re.search(' Q(.+?) ', txt).group(1)
            except:
                q = 'none'
                
            try: 
                year = re.search(' Q(.+?) (.+?) Earnings', txt).group(2)
            except:
                year = 'none'
                
            try: 
                ticker = re.search('\((.+?):(.+?)\)', txt).group(2)
            except:
                ticker = 'none'
            try:
                company_name = re.search('(.+?)\((.+?):', txt).group(1)
            except:
                company_name = 'none'
            try:
                date_time_str = re.search('Call (.+?) ET', txt).group(1)
                try:
                    date_time_obj = datetime.datetime.strptime(date_time_str, '%B %d, %Y %I:%M %p')
                except:
                    date_time_obj = 'none'
                try:
                    date = date_time_obj.date()
                except:
                    date = '00-00-00'
                try:
                    time = date_time_obj.time()
                except:
                    time = '00:00:00'
            except:
                date = '00-00-00'
                time = '00:00:00'
            companies.append({'cy_ticker': ticker, 'cy_name': company_name})
            conferences.append({'cf_id': conf_id, 'cf_title': title, 'cf_date': date, 'cf_time': time, 'Q': q, 'cf_year': year, 'cy_ticker': ticker})
           

        comp_participant = soup.find('p', {'class':'p p1'}, text = 'Company Participants')
      
        for sibling in comp_participant.next_siblings:
            try:
                if(sibling.text == 'Conference Call Participants'):
                    break
                p = sibling.text
                name = re.search('(.+?) - (.+?)', p).group(1)
                type_ = re.search('(.+?) - (.+)', p).group(2)
                organization = company_name
                participants.append({'p_name': name, 'p_type': type_, 'p_org': organization, 'cf_id': conf_id})
            except:
                continue
         
        conf_participant = soup.find('p', {'class':'p p1'}, text = 'Conference Call Participants')
        for sibling in conf_participant.next_siblings:
            try:
                p = sibling.text
                if(sibling.find('strong')):
                    now_speech = sibling
                    break
                name = re.search('(.+?) - (.+?)', p).group(1)
                organization = re.search('(.+?) - (.+)', p).group(2)
                type_ = 'guest participant'
                participants.append({'p_name': name, 'p_type': type_, 'p_org': organization, 'cf_id': conf_id})
            except:
                continue
        # get speech
        speaker = now_speech.text
        section = 'Presentation'
        for sibling in now_speech.next_siblings:
            try:
                if(sibling.find('strong')):
                    if(sibling.text == 'Question-and-Answer Session'):
                        section = 'Question-and-Answer'
                        continue
                    speaker = sibling.text
                    continue
                spoke = sibling.text
                spch_id += 1
                speech.append({'speech_id': spch_id, 'speech_section': section, 's_text': spoke, 'p_name': speaker, 'cf_id': conf_id})

            except:
                continue
    except:
        continue
        
write_to_company(companies)
write_to_conference(conferences)
write_to_participant(participants)
write_to_speech(speech)

